In [50]:
import pandas as pd
import sklearn
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import keras
from recsys_utils import *

ModuleNotFoundError: No module named 'recsys_utils'

In [21]:
#Call the Data
podcast_data = pd.read_csv("../Data/podcasts_data.csv")
podcast_data.head()


,Genre,Podcast Name,Description,Publisher,Total Episodes,Spotify URL
0,arts and entertainment,Easy Stories in English,"Learning a language is hard, but Easy Stories ...","Ariel Goodbody, Polyglot English Teacher & Gla...",215,https://open.spotify.com/show/23zdIqNUb0riR51w...
1,arts and entertainment,Podcast Buku Kutu,"EPISODE BARU SETIAP SENIN, RABU, dan JUMAT -- ...",Aditya Hadi - PODLUCK,162,https://open.spotify.com/show/3w5zKrbQ6kgB0RKI...
2,arts and entertainment,Podcast Resensi Buku,Kumpulan resensi beragam buku berbagai genre d...,Podcast Resensi Buku - PODLUCK,264,https://open.spotify.com/show/6woLsDl6CSntzeWU...
3,arts and entertainment,Join the Party,Join the Party is an actual play podcast with ...,Multitude,400,https://open.spotify.com/show/1kdF5OgsMBcK4IoU...
4,arts and entertainment,SupremeMasterTV,Supreme Master Television is an international ...,SupremeMasterTV,500,https://open.spotify.com/show/5bCgERRINgZWhauS...


In [30]:
#Slice the Column into Genre and Podcast Name
data_slice = podcast_data.iloc[:, :2]
print(data_slice.head())
print(data_slice.count())

                    Genre             Podcast Name
0  arts and entertainment  Easy Stories in English
1  arts and entertainment        Podcast Buku Kutu
2  arts and entertainment     Podcast Resensi Buku
3  arts and entertainment           Join the Party
4  arts and entertainment          SupremeMasterTV
Genre           2197
Podcast Name    2197
dtype: int64


In [48]:
# Divide data into Training data, Validation data and testing data

# Split data into 70% training and 30% remaining data
train_data, remaining_data = train_test_split(data_slice, test_size=0.3, random_state=42)

# Further split remaining data into 50% validation and 50% testing
# Since 15% of the original data is equivalent to 50% of the remaining data after the first split
validation_data, test_data = train_test_split(remaining_data, test_size=0.5, random_state=42)

# Display the sizes of the resulting datasets
print("Training data size:", len(train_data))
print("Validation data size:", len(validation_data))
print("Testing data size:", len(test_data))

Training data size: 1537
Validation data size: 330
Testing data size: 330


In [90]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

# Load your dataset
# Assuming your dataset contains columns: genre, podcast_name

# Preprocess and clean your data if necessary

# Feature Engineering
data_slice['features'] = data_slice['Genre'] + ' ' + data_slice['Podcast Name']

# Initialize TF-IDF vectorizer
tfidf_vectorizer = TfidfVectorizer()

# Fit and transform the text data
tfidf_matrix = tfidf_vectorizer.fit_transform(data_slice['features'])

# Compute cosine similarity between podcasts
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

# Function to get recommendations for a given podcast
def get_recommendations_by_genre(genre, n=10):
    # Convert the input genre to lowercase
    genre = genre.lower()
    
    # Filter the dataset to include only podcasts with the given genre (case-insensitive)
    genre_podcasts = data_slice[data_slice['Genre'].str.lower() == genre]

    # If there are podcasts with the given genre
    if not genre_podcasts.empty:
        # Get the top n recommended podcasts from the same genre
        recommendations = genre_podcasts['Podcast Name'].head(n)
        return recommendations
    else:
        return "No podcasts found with the given genre"

# Example: Get recommendations for a given genre (input in uppercase)
genre = 'celebrities'
recommendations = get_recommendations_by_genre(genre)
print(recommendations)

100    tl
101    tl
102    tl
103    fr
104    tr
105    tl
106    tl
107    tl
108    tl
109    tl
Name: Podcast Name, dtype: object


In [91]:
def get_recommendations_by_genre(genre, n=10):
    # Convert the input genre to lowercase
    genre = genre.lower()
    
    # Filter the dataset to include only podcasts with the given genre (case-insensitive)
    genre_podcasts = data_slice[data_slice['Genre'].str.lower() == genre]

    # If there are podcasts with the given genre
    if not genre_podcasts.empty:
        # Get the top n recommended podcasts from the same genre
        recommendations = genre_podcasts[['Genre']].head(n)
        return recommendations
    else:
        return pd.DataFrame(columns=['Genre'])  # Return an empty DataFrame

# Example: Get recommendations for a given genre (input in lowercase)
genre = 'celeb'
recommendations = get_recommendations_by_genre(genre)
print(recommendations)

Empty DataFrame
Columns: [Genre]
Index: []


In [92]:
import pandas as pd

# Function to get recommendations for a given genre
def get_recommendations_by_genre(genre, n=10):
    # Convert the input genre to lowercase
    genre = genre.lower()
    
    # Filter the dataset to include only podcasts with the given genre (case-insensitive)
    genre_podcasts = data_slice[data_slice['Genre'].str.lower() == genre]

    # If there are podcasts with the given genre
    if not genre_podcasts.empty:
        # Get the top n recommended podcasts from the same genre
        recommendations = genre_podcasts[['Podcast Name']].head(n)
        # Merge recommendations with the original dataset to get the full podcast information
        recommendations = recommendations.merge(data_slice, on='Podcast Name')
        return recommendations
    else:
        return pd.DataFrame(columns=data_slice.columns)  # Return an empty DataFrame with all columns

# Example: Get recommendations for a given genre (input in lowercase)
genre = 'celebrities'
recommendations = get_recommendations_by_genre(genre)
print(recommendations)


      Podcast Name                   Genre language                   features
0               tl  arts and entertainment       en  arts and entertainment tl
1               tl  arts and entertainment       en  arts and entertainment tl
2               tl  arts and entertainment       en  arts and entertainment tl
3               tl  arts and entertainment       en  arts and entertainment tl
4               tl  arts and entertainment       en  arts and entertainment tl
...            ...                     ...      ...                        ...
11463           tl              true crime       en              true crime tl
11464           tl              true crime       en              true crime tl
11465           tl              true crime       en              true crime tl
11466           tl              true crime       en              true crime tl
11467           tl              true crime       en              true crime tl

[11468 rows x 4 columns]


In [17]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, Flatten, Dense, Concatenate

# Load the dataset
data = pd.read_csv('../Data/podcasts_data.csv')

# Encode the "Genre" and "Podcast Name" columns
genre_encoder = LabelEncoder()
podcast_encoder = LabelEncoder()

data['Genre'] = genre_encoder.fit_transform(data['Genre'])
data['Podcast Name'] = podcast_encoder.fit_transform(data['Podcast Name'])

# Define the number of unique genres and podcasts
num_genres = data['Genre'].nunique()
num_podcasts = data['Podcast Name'].nunique()

# Neural Network Model
def create_model(num_genres, num_podcasts, embedding_size=50):
    # Genre input
    genre_input = Input(shape=(1,), name='genre_input')
    genre_embedding = Embedding(input_dim=num_genres, output_dim=embedding_size, name='genre_embedding')(genre_input)
    genre_vec = Flatten(name='genre_flatten')(genre_embedding)

    # Podcast input
    podcast_input = Input(shape=(1,), name='podcast_input')
    podcast_embedding = Embedding(input_dim=num_podcasts, output_dim=embedding_size, name='podcast_embedding')(podcast_input)
    podcast_vec = Flatten(name='podcast_flatten')(podcast_embedding)

    # Concatenate features
    concat = Concatenate(name='concat')([genre_vec, podcast_vec])
    dense = Dense(128, activation='relu', name='dense_1')(concat)
    dense = Dense(64, activation='relu', name='dense_2')(dense)
    output = Dense(1, activation='sigmoid', name='output')(dense)

    model = Model(inputs=[genre_input, podcast_input], outputs=output)
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

    return model

# Create the model
model = create_model(num_genres, num_podcasts)

# Prepare the data for training
genres = data['Genre'].values
podcasts = data['Podcast Name'].values

# Assume interaction is 1 (genre matches podcast)
labels = np.ones(len(genres))

# Train the model
model.fit([genres, podcasts], labels, epochs=10, batch_size=32, validation_split=0.2)

# Making recommendations
# For a given podcast, find similar podcasts
def recommend_podcasts(podcast_name, top_n=5):
    podcast_id = podcast_encoder.transform([podcast_name])[0]
    all_genres = np.arange(num_genres)
    podcast_vec = np.array([podcast_id] * num_genres)
    predictions = model.predict([all_genres, podcast_vec])
    top_podcasts = np.argsort(predictions.flatten())[-top_n:][::-1]
    return podcast_encoder.inverse_transform(top_podcasts)

# Example recommendation for "Podcast A"
recommended_podcasts = recommend_podcasts('books', top_n=5)
print("Recommended Podcasts:", recommended_podcasts)

Epoch 1/10
55/55 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.9364 - loss: 0.5057 - val_accuracy: 1.0000 - val_loss: 0.0941
Epoch 2/10
55/55 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 1.0000 - loss: 0.0013 - val_accuracy: 1.0000 - val_loss: 0.0629
Epoch 3/10
55/55 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 1.0000 - loss: 2.9100e-04 - val_accuracy: 1.0000 - val_loss: 0.0593
Epoch 4/10
55/55 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 1.0000 - loss: 1.8670e-04 - val_accuracy: 1.0000 - val_loss: 0.0564
Epoch 5/10
55/55 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 1.0000 - loss: 1.2771e-04 - val_accuracy: 1.0000 - val_loss: 0.0542
Epoch 6/10
55/55 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 1.0000 - loss: 9.5127e-05 - val_accuracy: 1.0000 - val_loss: 0.0523
Epoch 7/10
55/55 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 1.0000 - loss: 7.2127e-05 - val_accuracy: 1.0000 - val_loss: 0.0508
Epoch 8/10
55/55 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 1.0000 - loss: 5.8283e-05 - val_accu

ValueError: y contains previously unseen labels: 'books'

### FIX CODE 1

In [37]:
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Load your dataset
data = pd.read_csv('../Data/podcasts_data.csv')
data = data.iloc[:, :2]
data.head()


,Genre,Podcast Name
0,arts and entertainment,Easy Stories in English
1,arts and entertainment,Podcast Buku Kutu
2,arts and entertainment,Podcast Resensi Buku
3,arts and entertainment,Join the Party
4,arts and entertainment,SupremeMasterTV


In [38]:
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\syari\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [39]:
# Cleaning
def clean_text(text):
    text = text.lower()  # Convert to lowercase
    text = re.sub(r'[^\w\s]', '', text)  # Remove punctuation
    text = re.sub(r'\d+', '', text)  # Remove numbers
    text = re.sub(r'\s+', ' ', text)  # Remove extra whitespace
    text = text.strip()  # Remove leading and trailing whitespace
    stop_words = set(stopwords.words('english'))
    text = ' '.join(word for word in text.split() if word not in stop_words)  # Remove stop words
    return text

data['Podcast Name'] = data['Podcast Name'].apply(clean_text)

In [40]:
# Tokenize the text data
tokenizer = Tokenizer()
tokenizer.fit_on_texts(data['Podcast Name'])

# Convert text to sequences
sequences = tokenizer.texts_to_sequences(data['Podcast Name'])

# Pad the sequences to ensure uniform input size
maxlen = 50  # Choose an appropriate maximum length for padding
X = pad_sequences(sequences, maxlen=maxlen)

# Encode the labels
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(data['Genre'])

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [41]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout


# Define the model
vocab_size = len(tokenizer.word_index) + 1  # Size of the vocabulary
embedding_dim = 100

model = Sequential([
    Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=maxlen),
    LSTM(128, return_sequences=True),
    Dropout(0.5),
    LSTM(128),
    Dropout(0.5),
    Dense(64, activation='relu'),
    Dropout(0.5),
    Dense(len(label_encoder.classes_), activation='softmax')
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

model.summary()


d:\Capstone\.conda\Lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential_10"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_10 (Embedding)        │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_20 (LSTM)                  │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_30 (Dropout)            │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_21 (LSTM)                  │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_31 (Dropout)            │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_20 (Dense)                │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_32 (Dropout)            │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_21 (Dense)                │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [42]:
# Train the model
history = model.fit(X_train, y_train, epochs=30, batch_size=32, validation_split=0.2)


Epoch 1/30
44/44 ━━━━━━━━━━━━━━━━━━━━ 5s 41ms/step - accuracy: 0.0271 - loss: 3.9522 - val_accuracy: 0.0114 - val_loss: 3.9461
Epoch 2/30
44/44 ━━━━━━━━━━━━━━━━━━━━ 2s 35ms/step - accuracy: 0.0219 - loss: 3.9361 - val_accuracy: 0.0256 - val_loss: 3.9349
Epoch 3/30
44/44 ━━━━━━━━━━━━━━━━━━━━ 2s 37ms/step - accuracy: 0.0192 - loss: 3.9273 - val_accuracy: 0.0227 - val_loss: 3.9209
Epoch 4/30
44/44 ━━━━━━━━━━━━━━━━━━━━ 2s 38ms/step - accuracy: 0.0161 - loss: 3.9098 - val_accuracy: 0.0142 - val_loss: 3.9102
Epoch 5/30
44/44 ━━━━━━━━━━━━━━━━━━━━ 2s 35ms/step - accuracy: 0.0312 - loss: 3.8893 - val_accuracy: 0.0227 - val_loss: 3.9007
Epoch 6/30
44/44 ━━━━━━━━━━━━━━━━━━━━ 2s 38ms/step - accuracy: 0.0312 - loss: 3.8616 - val_accuracy: 0.0568 - val_loss: 3.8454
Epoch 7/30
44/44 ━━━━━━━━━━━━━━━━━━━━ 1s 33ms/step - accuracy: 0.0487 - loss: 3.7986 - val_accuracy: 0.0653 - val_loss: 3.7403
Epoch 8/30
44/44 ━━━━━━━━━━━━━━━━━━━━ 2s 36ms/step - accuracy: 0.0741 - loss: 3.6273 - val_accuracy: 0.0881 - v

In [24]:
# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Accuracy: {accuracy*100:.2f}%')




14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.1501 - loss: 5.3691
Accuracy: 14.77%
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
parenting


In [25]:
# Function to predict the genre of a given podcast name
def predict_genre(podcast_name):
    sequence = tokenizer.texts_to_sequences([podcast_name])
    padded_sequence = pad_sequences(sequence, maxlen=maxlen)
    prediction = model.predict(padded_sequence)
    genre = label_encoder.inverse_transform([prediction.argmax()])[0]
    return genre

# Example usage
print(predict_genre("Career"))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
parenting


### TESTING CODE

In [28]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout

# Verify the tokenizer and maxlen
texts = ["Career", "Easy Stories"]  # Example text data
tokenizer = Tokenizer()
tokenizer.fit_on_texts(texts)
vocab_size = len(tokenizer.word_index) + 1  # Size of the vocabulary
maxlen = 50  # Maximum length of sequences (should match the length used in preprocessing)

# Define the model
model = Sequential([
    Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=maxlen),
    LSTM(128, return_sequences=True),
    Dropout(0.5),
    LSTM(128),
    Dropout(0.5),
    Dense(64, activation='relu'),
    Dropout(0.5),
    Dense(len(label_encoder.classes_), activation='softmax')  # Ensure label_encoder is properly set
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Print the model summary
model.summary()


Model: "sequential_6"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_6 (Embedding)         │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_12 (LSTM)                  │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_18 (Dropout)            │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_13 (LSTM)                  │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_19 (Dropout)            │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_12 (Dense)                │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_20 (Dropout)            │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_13 (Dense)                │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [29]:
import tensorflow as tf
print(tf.__version__)

2.16.1


In [30]:
print(f"Vocabulary Size: {vocab_size}")
print(f"Max Length: {maxlen}")

# Example padded sequences
sequences = tokenizer.texts_to_sequences(texts)
padded_sequences = pad_sequences(sequences, maxlen=maxlen)
print(f"Padded Sequences Shape: {padded_sequences.shape}")


Vocabulary Size: 4
Max Length: 50
Padded Sequences Shape: (2, 50)


In [31]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder
import numpy as np
import re
from nltk.corpus import stopwords
import nltk

nltk.download('stopwords')

# Example text data
texts = ["Sample text data", "Another example sentence"]

# Fit tokenizer on text data
tokenizer = Tokenizer()
tokenizer.fit_on_texts(texts)

# Verify tokenizer configuration
vocab_size = len(tokenizer.word_index) + 1
print(f"Vocabulary Size: {vocab_size}")

# Generate and pad sequences
sequences = tokenizer.texts_to_sequences(texts)
maxlen = 50
padded_sequences = pad_sequences(sequences, maxlen=maxlen)
print(f"Padded Sequences Shape: {padded_sequences.shape}")

# Mockup label encoding for the example
labels = ['genre1', 'genre2']
label_encoder = LabelEncoder()
label_encoder.fit(labels)
y = label_encoder.transform(['genre1', 'genre2'])

# Define the model
model = Sequential([
    Embedding(input_dim=vocab_size, output_dim=100, input_length=maxlen),
    LSTM(128, return_sequences=True),
    Dropout(0.5),
    LSTM(128),
    Dropout(0.5),
    Dense(64, activation='relu'),
    Dropout(0.5),
    Dense(len(label_encoder.classes_), activation='softmax')
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Print the model summary
model.summary()


Vocabulary Size: 7
Padded Sequences Shape: (2, 50)


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\syari\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
d:\Capstone\.conda\Lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential_7"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_7 (Embedding)         │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_14 (LSTM)                  │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_21 (Dropout)            │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_15 (LSTM)                  │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_22 (Dropout)            │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_14 (Dense)                │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_23 (Dropout)            │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_15 (Dense)                │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [32]:
# Function to preprocess the podcast name
def preprocess_podcast_name(podcast_name):
    podcast_name = clean_text(podcast_name)
    sequence = tokenizer.texts_to_sequences([podcast_name])
    padded_sequence = pad_sequences(sequence, maxlen=maxlen)
    return padded_sequence

# Generate embeddings for the entire dataset
def get_embeddings(model, data):
    embedding_layer = model.layers[0]
    embedding_model = tf.keras.Model(inputs=model.input, outputs=embedding_layer.output)
    return embedding_model.predict(data)

# Get embeddings for all podcast names
all_podcast_embeddings = get_embeddings(model, X)

# Get embedding for the input podcast name
def get_podcast_embedding(podcast_name):
    processed_name = preprocess_podcast_name(podcast_name)
    embedding_model = tf.keras.Model(inputs=model.input, outputs=model.layers[0].output)
    return embedding_model.predict(processed_name)

input_podcast_embedding = get_podcast_embedding("Easy Stories in English")

# Calculate cosine similarity
def find_similar_podcasts(input_embedding, all_embeddings, top_n=5):
    similarities = cosine_similarity(input_embedding, all_embeddings)
    similar_indices = similarities.argsort()[0][-top_n:][::-1]  # Get indices of top N similar podcasts
    return similar_indices

similar_indices = find_similar_podcasts(input_podcast_embedding, all_podcast_embeddings)

# Get the most similar podcast names
similar_podcast_names = data['Podcast Name'].iloc[similar_indices]
print(similar_podcast_names)

ValueError: The layer sequential_7 has never been called and thus has no defined input.